# Train and Predict Drones

### Abstract

...


### Introduction

...


### Dataset



# 🧠 1. Setup

pip install `ultralytics` and [dependencies](https://github.com/ultralytics/ultralytics/blob/main/pyproject.toml) and check software and hardware.

[![PyPI - Version](https://img.shields.io/pypi/v/ultralytics?logo=pypi&logoColor=white)](https://pypi.org/project/ultralytics/) [![PyPI - Python Version](https://img.shields.io/pypi/pyversions/ultralytics?logo=python&logoColor=gold)](https://pypi.org/project/ultralytics/)

### 1.0 Importing Libraries

In [ ]:
!pip install ultralytics markdown rich wrapt -q

In [ ]:
from IPython.display import display, Image as IPyImage
from collections import defaultdict
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from ultralytics import YOLO

import pandas as pd
import ultralytics
import numpy as np
import datetime
import hashlib
import random
import shutil
import json
import math
import re
import os

ultralytics.checks()

### 1.1 Global Definitions

In [ ]:
DATASET_NAME_DIR = './datasets/new_dataset/'
TRAINING_DATASET_DIRECTORY = DATASET_NAME_DIR + 'training/'
VALIDATION_DATASET_DIRECTORY = DATASET_NAME_DIR + 'valid/'
MODELS_DIRECTORY = './models/'
RUNS_DIRECTORY = "./runs"

# 📂 2. Dataset

### 2.1 Acquire Dataset

Download dataset from hugging face.

### 2.2 Data Structure Check

##### Directory path validation

In [ ]:
train_image_names = [f for f in os.listdir(TRAINING_DATASET_DIRECTORY) if f.endswith(".jpg") or f.endswith(".JPEG")]
val_image_names = [f for f in os.listdir(VALIDATION_DATASET_DIRECTORY) if f.endswith(".jpg") or f.endswith(".JPEG")]

print(f"Number of images in training folder: {len(train_image_names)}")
print(f"Number of images in valid folder: {len(val_image_names)}")

train_txt_names = [f for f in os.listdir(TRAINING_DATASET_DIRECTORY) if f.endswith(".txt")]
val_txt_names = [f for f in os.listdir(VALIDATION_DATASET_DIRECTORY) if f.endswith(".txt")]

print(f"\nNumber of TXT files in train labels folder: {len(train_txt_names)}")
print(f"Number of TXT files in val labels folder: {len(val_txt_names)}")

##### Duplicate files

Check for duplicate images and labels.

In [ ]:
paths_to_check = [
    [os.path.join(TRAINING_DATASET_DIRECTORY, f) for f in train_image_names],
    [os.path.join(VALIDATION_DATASET_DIRECTORY, f) for f in val_image_names],
    [os.path.join(TRAINING_DATASET_DIRECTORY, f) for f in train_txt_names],
    [os.path.join(VALIDATION_DATASET_DIRECTORY, f) for f in val_txt_names],
]


def sha3_file(path, chunk_size=8192):
    hash_sha3 = hashlib.sha3_256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(chunk_size), b""):
            hash_sha3.update(chunk)
    return hash_sha3.hexdigest()


duplicates_array = []

for path in paths_to_check:

    # Build list of (hash, path)
    pairs = [
        (sha3_file(f), f)
        for f in path
    ]

    # Group paths by hash
    hash_map = defaultdict(list)
    for h, p in pairs:
        hash_map[h].append(p)

    # Extract only duplicates
    duplicates = {h: paths for h, paths in hash_map.items() if len(paths) > 1}
    duplicates_array.append(duplicates)

    # Print results
    if duplicates:
        print("Duplicate files found:")
        for h, paths in duplicates.items():
            print(f"Hash: {h}")
            for p in paths:
                print(f"  - {p}")
    else:
        print(f"No duplicate files found in the scanned directories.")


##### ⚠️Delete duplicate files ⚠️

Delete for duplicate images and labels.

In [ ]:
try:
    if input("Files are going to be deleted. Write yes to continue.") == 'yes':
        for duplicates in duplicates_array:
            for paths in duplicates.values():
                for path in paths:
                    os.remove(path)
except KeyboardInterrupt:
    print("Deletion aborted.")

##### Check for orphelins txt file

In [ ]:
paths_to_check = [
    [os.path.join(TRAINING_DATASET_DIRECTORY, f) for f in train_txt_names],
    [os.path.join(VALIDATION_DATASET_DIRECTORY, f) for f in val_txt_names],
]

orphelins_label = []

for path_list in paths_to_check:
    for p in path_list:

        root, _ = os.path.splitext(p)

        candidates = [
            root + ".jpg",
            root + ".JPG",
            root + ".jpeg",
            root + ".JPEG",
            root + ".png",
            root + ".PNG",
        ]

        if not any(os.path.isfile(c) for c in candidates):
            orphelins_label.append(p)
            print(f"TXT file {p} does not belong to any image.")


##### ⚠️Delete orphelin labels files ⚠️

Delete labels that don't belong to any images.

In [ ]:
try:
    if input("Files are going to be deleted. Write yes to continue.") == 'yes':
        for orphelin_path in orphelins_label:
            os.remove(orphelin_path)
except KeyboardInterrupt:
    print("Deletion aborted.")

##### Rename files

Rename images and labels from 0 to n images.

In [ ]:
TRAIN = TRAINING_DATASET_DIRECTORY
VAL = VALIDATION_DATASET_DIRECTORY

jpeg_exts = [".jpg", ".jpeg", ".JPG", ".JPEG"]

tmp = os.path.join(DATASET_NAME_DIR, "tmp")


def numeric_key(name):
    nums = re.findall(r'\d+', name)
    return int(nums[0]) if nums else float('inf')


def collect_images(folder):
    return sorted(
        [f for f in os.listdir(folder) if f.endswith(tuple(jpeg_exts))],
        key=numeric_key
    )


def process(folder, start_i, out_dir):
    files = collect_images(folder)

    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    for f in files:
        root, ext = os.path.splitext(f)

        # find real existing image
        image_path = None
        for e in jpeg_exts:
            p = os.path.join(folder, root + e)
            if os.path.isfile(p):
                image_path = p
                break

        if image_path is None:
            print("Missing image for:", f)
            continue

        label_path = os.path.join(folder, root + ".txt")
        if not os.path.isfile(label_path):
            os.rename(image_path, os.path.join(out_dir, f"{start_i}.jpg"))
            start_i += 1
            continue

        new_image = os.path.join(out_dir, f"{start_i}.jpg")
        new_label = os.path.join(out_dir, f"{start_i}.txt")

        # Move safely
        os.rename(image_path, new_image)
        os.rename(label_path, new_label)

        start_i += 1

    return start_i


# === RUN ===

i = 0
i = process(TRAIN, i, tmp + '_train')
i = process(VAL, i, tmp + '_valid')

print(f"DONE — All files renamed safely into {DATASET_NAME_DIR}tmp_NAME")

## Set labels

In [ ]:
!pip install torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
!pip install sam3 opencv-python pandas scikit-learn requests huggingface_hub ipywidgets

In [ ]:
from sam3.model.sam3_image_processor import Sam3Processor
from sam3.visualization_utils import plot_results
from sam3 import build_sam3_image_model
from huggingface_hub import login

In [ ]:
login(os.getenv("HF_TOKEN"))
bpe_name = "bpe_simple_vocab_16e6.txt.gz"

##### Load model

In [ ]:
model = build_sam3_image_model(bpe_path=os.path.join(MODELS_DIRECTORY, bpe_name))
processor = Sam3Processor(model)

In [ ]:
image_path = f"datasets/new_dataset/training/633.jpg"
prompt = "drone"
confidence_threshold = 0.5

image = Image.open(image_path)
width, height = image.size
processor = Sam3Processor(model, confidence_threshold=confidence_threshold)
inference_state = processor.set_image(image)

processor.reset_all_prompts(inference_state)
inference_state = processor.set_text_prompt(state=inference_state, prompt=prompt)
img0 = Image.open(image_path)
plot_results(img0, inference_state)

##### Settings

In [ ]:
labelize_overwrite_label: bool = True
labelize_prompt: str = 'drone'
labelize_confidence_threshold: float = 0.62
labelize_save_inference: bool = False
labelize_drone_class = 0
labelize_output_label_file_directory = os.path.join(DATASET_NAME_DIR, 'new_labels')
labelize_output_label_file_errors = os.path.join(labelize_output_label_file_directory, 'errors.txt')
labelize_output_label_file_success = os.path.join(labelize_output_label_file_directory, 'success.txt')

paths_to_labelize = [
    [os.path.join(TRAINING_DATASET_DIRECTORY, f) for f in train_image_names],
    [os.path.join(VALIDATION_DATASET_DIRECTORY, f) for f in val_image_names],
]

In [ ]:
os.makedirs(labelize_output_label_file_directory, exist_ok=True)

image_failed_labelize = []

for paths in paths_to_labelize:
    for image_path in paths:
        root, ext = os.path.splitext(image_path)
        root_label_name = root.split("/")[-1]
        current_label_path = root + ".txt"
        saved_label_path = os.path.join(labelize_output_label_file_directory, root_label_name) + ".txt"

        if os.path.isfile(current_label_path) and not labelize_overwrite_label:
            continue

        if os.path.isfile(saved_label_path) and not labelize_overwrite_label:
            continue

        # Load image
        image = Image.open(image_path)
        if image.mode != "RGB":
            with open(labelize_output_label_file_errors, "a") as f:
                image_failed_labelize.append(image_path)
                f.write(f"{image_path} is not RGB.\n")
            continue
        width, height = image.size

        # Run SAM3 processor
        processor = Sam3Processor(model, confidence_threshold=labelize_confidence_threshold)
        inference_state = processor.set_image(image)
        processor.reset_all_prompts(inference_state)
        inference_state = processor.set_text_prompt(state=inference_state, prompt=labelize_prompt)

        # --- Save YOLO labels ---
        bboxes = inference_state.get("boxes", None)  # fixed

        if bboxes is None or len(bboxes) == 0:
            # print(f"No detections for {image_path}")
            image_failed_labelize.append(image_path)
            with open(labelize_output_label_file_errors, "a") as f:
                f.write(f"No detections for {image_path}\n")
            continue

        yolo_lines = []
        for box in bboxes:
            x_min, y_min, x_max, y_max = box  # SAM3 gives XYXY

            # Convert to YOLO format
            bbox_width = x_max - x_min
            bbox_height = y_max - y_min

            cx = (x_min + bbox_width / 2) / width
            cy = (y_min + bbox_height / 2) / height

            nw = bbox_width / width
            nh = bbox_height / height

            yolo_lines.append(
                f"{labelize_drone_class} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}"
            )

        with open(saved_label_path, "w") as f:
            f.write("\n".join(yolo_lines))
        with open(labelize_output_label_file_success, "a") as f:
            f.write(f"Saved YOLO labels: {saved_label_path}\n")

    with open(labelize_output_label_file_success, 'r') as fp:
        nb_lines = len(fp.readlines())
        print(f"Successfully procedded files: {nb_lines} out of {len(paths)} for {paths[0][0]}")

In [ ]:
try:
    if input("Files are going to be deleted. Write yes to continue.") == 'yes':
        for image_path in image_failed_labelize:
            os.remove(image_path)
except KeyboardInterrupt:
    print("Deletion aborted.")

### 2.3 Check

Load N random images and show rectangle round the drone.

In [ ]:
def load_image_with_boxes(image_path, label_path):
    """Returns a PIL image with YOLO bounding boxes drawn."""
    img = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(img)

    w, h = img.size

    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            lines = f.readlines()

        for line in lines:
            cls, xc, yc, bw, bh = map(float, line.split())

            # YOLO normalized → pixel coordinates
            x_center = xc * w
            y_center = yc * h
            box_width = bw * w
            box_height = bh * h

            x1 = x_center - box_width / 2
            y1 = y_center - box_height / 2
            x2 = x_center + box_width / 2
            y2 = y_center + box_height / 2

            draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
            draw.text((x1, y1), f"{int(cls)}", fill="red")

    return img

In [ ]:
N = 48  # number of random samples
cols = 4

# list all images
image_paths = [p for p in os.listdir(TRAINING_DATASET_DIRECTORY) if p.lower().endswith((".jpg", ".png"))]
sampled_images = random.sample(image_paths, min(N, len(image_paths)))

# -------- DISPLAY GRID -------- #

rows = math.ceil(len(sampled_images) / cols)

fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 2))
axes = axes.flatten()

for ax, img_name in zip(axes, sampled_images):
    img_path = os.path.join(TRAINING_DATASET_DIRECTORY, img_name)
    label_path = os.path.join(TRAINING_DATASET_DIRECTORY, img_name.rsplit(".", 1)[0] + ".txt")

    img = load_image_with_boxes(img_path, label_path)

    ax.imshow(img)
    ax.axis("off")
    ax.set_title(img_name, fontsize=8)

# turn off unused axes
for ax in axes[len(sampled_images):]:
    ax.axis("off")

plt.tight_layout()
plt.show()

# ⚙️ 3. Training

Purpose: Handle model setup and training configuration.

### 3.1 Model Selection

In [ ]:
YOLO_MODELS = {
    "nano": "yolo11n.pt",
    "small": "yolo11s.pt",
    "medium": "yolo11m.pt",
    "large": "yolo11l.pt",
    "xlarge": "yolo11x.pt",
}

selected_size = "nano"
model = YOLO(MODELS_DIRECTORY + YOLO_MODELS[selected_size])

### 3.2 Training Configuration
Define hyperparameters (epochs, batch size, image size)

In [ ]:
import yaml

data_yaml = dict(
    train=os.path.join('../../', TRAINING_DATASET_DIRECTORY),
    val=os.path.join('../../', VALIDATION_DATASET_DIRECTORY),
    nc=1,
    names=['drone']
)

data_config_path = os.path.join(DATASET_NAME_DIR, 'data.yaml')

with open(data_config_path, 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

%cat "$data_config_path"

In [ ]:
train_config = {
    'data': data_config_path,
    'epochs': 50,
    'imgsz': 640,
    'batch': 16,
    'lr0': 0.001,
    'optimizer': 'SGD',
    'project': RUNS_DIRECTORY
}

train_config

**Parameter breakdown:**

- **train**: Executes the YOLOv11x training pipeline.
- **model**=yolov11x.pt: Uses pre-trained YOLOv11x weights as initialization.
- **data**=/content/data.yaml: Specifies the dataset configuration file.
- **imgsz**=640: Sets input resolution to enhance small-object detection.
- **lr0**=0.001: Sets the learing rate for each training step.
- **epochs**=32: Defines the number of training cycles over the dataset.
- **batch**=16: Sets the batch size for each training step.
- **device**=0: Allocates GPU device 0 for training.
- **optimizer**=AdamW: As default, AdamW optimization algorithm utilized.

### 3.3 Run Training

In [ ]:
results = model.train(**train_config)
plot_results(os.path.join(RUNS_DIRECTORY, "/train/results.csv"))

### 3.4 Result of training

In [ ]:
display(IPyImage(filename=os.path.join(RUNS_DIRECTORY, "/train/results.png")))

In [ ]:
results_paths = []

for root, _, files in os.walk(os.path.join(RUNS_DIRECTORY, "train")):
    for f in files:
        if f.lower().endswith((".jpg")):
            results_paths.append(os.path.join(root, f))
results_paths = sorted(results_paths)

In [ ]:
%matplotlib inline

for path in results_paths:
    image = np.array(Image.open(path))
    plt.figure(figsize=(20, 10))
    plt.imshow(image)
    plt.axis("off")
    plt.show()


### 3.5 Evaluate Model


In [ ]:
metrics = model.val()
metrics

### 3.6 Compare with previous trains

In [ ]:
from datetime import datetime
saved_train_path = "./saved_trains"

# List all folders in the saved_train_path
folders_results = [dir for dir in os.listdir(saved_train_path) if os.path.isdir(os.path.join(saved_train_path, dir))]

# Convert folder names to datetime objects for sorting
folders_results_sorted = sorted(
    folders_results,
    key=lambda x: datetime.strptime(x, "%d-%m-%Y_%H:%M:%S"),
    reverse=True  # latest first
)

# Take the last 2 trainings by date
last_two_folders = folders_results_sorted[:2]

# Dictionary to store the last row of each folder
results_data = {}

for folder in last_two_folders:
    path = os.path.join(saved_train_path, folder)
    results_csv = os.path.join(path, "results.csv")

    if os.path.exists(results_csv):
        df = pd.read_csv(results_csv)
        # Get the last row as a dictionary
        last_row = df.iloc[-1].to_dict()

        # Compute a custom YOLO train score
        score = (
            last_row['metrics/mAP50-95(B)'] * 0.7 +
            last_row['metrics/recall(B)'] * 0.2 -
            last_row['val/box_loss'] * 0.1
        )

        # Add the score to the dictionary
        last_row['train_score'] = score

        results_data[folder] = last_row

# Convert the dictionary to a pandas DataFrame
summary_df = pd.DataFrame.from_dict(results_data, orient='index')
summary_df.reset_index(inplace=True)
summary_df.rename(columns={'index': 'folder'}, inplace=True)

# Sort by train_score if desired
best_train = summary_df.sort_values(by='train_score', ascending=False)

# 🔍 4. Prediction & Visualization

Purpose: Evaluate results qualitatively and quantitatively.


### 4.1 Run Predictions

In [ ]:
best_path = os.path.join(RUNS_DIRECTORY, "train/weights/best.pt")

In [ ]:
custom_model = YOLO(best_path)
!rm -rf ./runs/predict
preds = custom_model.predict(VALIDATION_DATASET_DIRECTORY, save=True, project=RUNS_DIRECTORY)

### 4.2 Result of Prediction

In [ ]:
results = custom_model.predict(VALIDATION_DATASET_DIRECTORY, conf=0.2)
print(len(results))

### 4.3 Visualize Predictions

In [ ]:
predictions_paths = []

for root, _, files in os.walk(os.path.join(RUNS_DIRECTORY, "predict")):
    for f in files:
        if f.lower().endswith((".jpg", ".png", ".jpeg")):
            predictions_paths.append(os.path.join(root, f))

In [ ]:
cols = 4
max_images_preview = -1
rows = math.ceil(len(predictions_paths) / cols)

plt.figure(figsize=(cols * 3, rows * 2))  # dynamic size
for i, path in enumerate(predictions_paths):
    if max_images_preview != -1 and max_images_preview < len(predictions_paths):
        break
    img = Image.open(path)
    plt.subplot(rows, cols, i + 1)
    plt.imshow(img)
    plt.axis("off")

plt.tight_layout()
plt.show()

# 📊 5. Analysis & Improvements
Purpose: Interpret and iterate.

### 5.1 Metrics Review

In [ ]:
metrics.box.map, metrics.box.map50, metrics.box.map75

### 5.2 Error Analysis

In [ ]:
metrics.confusion_matrix.plot()

### 5.3 Compare with previous trains

# ✅ 6. Export & Deployment

### 6.1 Save results

In [ ]:
source = os.path.join(RUNS_DIRECTORY, "train")
saved_success_file_name = ".saved.success"

check_for_file = os.path.isfile(os.path.join(source, saved_success_file_name))

if check_for_file:
    print("Training already saved. If you want to save it again, delete the .saved.success file.")
else:
    now = datetime.now().strftime('%d-%m-%Y_%H:%M:%S')
    destination = "./saved_trains/" + now

    shutil.copytree(source, destination, dirs_exist_ok=True)

    train_config_path = os.path.join(destination, 'train_config.json')
    with open(train_config_path, 'w') as outfile:
        yaml.dump(json.dumps(train_config), outfile, default_flow_style=True)

    saved_file = os.path.join(source, '.saved.success')

    with open(saved_file, "a") as f:
        f.write(now)
    print("Training saved successfully.")